In [2]:
import pandas as pd
import re
import csv
from nltk import word_tokenize,pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk import stem
from nltk import sent_tokenize


import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline
doc = nlp("Barack Obama was born in Hawaii.") # run annotation over a sentence
from sklearn.model_selection import train_test_split

2024-07-10 15:44:29 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/resources.json
2024-07-10 15:44:29 INFO: Downloading default packages for language: en (English) ...


2024-07-10 15:44:44 INFO: Downloaded file to /Users/gbaldonado/stanza_resources/en/default.zip


##### Stanford Core NLP
* We need to download the [Stanford core nlp](https://stanfordnlp.github.io/CoreNLP/download.html) package before we can start with the dataset creation process.  
``
cd /Users/krishns18/SFSU/Modeling_ALMA/corenlp/stanford-corenlp-4.0.0
java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer
``   
* The server will start and the command prompt will display like below   
``
(base) Shaileshs-MBP:stanford-corenlp-4.0.0 krishns18$ java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer
[main] INFO CoreNLP - --- StanfordCoreNLPServer#main() called ---
[main] INFO CoreNLP - Warning: cannot find edu/stanford/nlp/models/srparser/englishSR.ser.gz
[main] INFO CoreNLP - Setting default constituency parser to PCFG parser: edu/stanford/nlp/models/lexparser/englishPCFG.ser.gz
[main] INFO CoreNLP - To use shift reduce parser download English models jar from:
[main] INFO CoreNLP - https://stanfordnlp.github.io/CoreNLP/download.html
[main] INFO CoreNLP -     Threads: 16
[main] INFO CoreNLP - Starting server...
[main] INFO CoreNLP - StanfordCoreNLPServer listening at /0:0:0:0:0:0:0:0:9000
``

In [34]:
nlp = StanfordCoreNLP('http://localhost:9000')

In [4]:
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

**Read the essays**

In [5]:
# Read the essays and create a dataframe
df = pd.concat(pd.read_excel('data/annotated_xlsx/Attainment_CapitalsPilot_Fall2019.xlsx', sheet_name=None),ignore_index=True,sort=False)

In [6]:
df.shape

(570, 4)

In [7]:
df.head()

,ID CODE,Essay : Why am I here,Attainment - FINAL,First Gen - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...,0
1,pilot_study_02,I want to stay in my community & help educate ...,0,0
2,pilot_study_03,I am here to provide myself with opportunities...,0,0
3,pilot_study_04,I am here to figure out what I want to do with...,0,0
4,pilot_study_05,I am here at San Francisco State because I wan...,0,0


**Creating a separate dataset for Attainment**

In [8]:
attainment_df = df.copy()

In [9]:
# dropping the first gen column from attainment dataset.
attainment_df = attainment_df.drop(columns=['First Gen - FINAL'])
attainment_df.head()

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
1,pilot_study_02,I want to stay in my community & help educate ...,0
2,pilot_study_03,I am here to provide myself with opportunities...,0
3,pilot_study_04,I am here to figure out what I want to do with...,0
4,pilot_study_05,I am here at San Francisco State because I wan...,0


In [10]:
# drop essays which do not contain attainment
attainment_df = attainment_df[~attainment_df['Attainment - FINAL'].isin([0, '-', 'Unable to code'])]

In [11]:
attainment_df = attainment_df.dropna() # drop null

In [12]:
attainment_df.shape

(185, 3)

In [13]:
attainment_df

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
8,pilot_study_09,I’m here to learn and to be on the path toward...,I’m here to learn and to be on the path toward...
9,pilot_study_10,I am in SFSU because I want to go to medical s...,I am in SFSU because I want to go to medical s...
11,pilot_study_12,I am here in SFSU because I am the first gener...,My goal is to graduate with a degree B.A./B.S.
13,pilot_study_14,I am here because of my goal in trying to get ...,I am here because of my goal in trying to get ...
...,...,...,...
551,F19.PHYS232.07.23,I'm here because of prerequisite for my comput...,And my dream job is to be a software engineer.
559,F19.PHYS232.09.08,The reason I am here is to do experiments and ...,I am an aspiring Engineer and I love doing ex...
561,F19.PHYS232.09.10,I am in college to further my education. There...,I am studying to become an electrical engineer.
562,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...


In [14]:
# total number of essays
print("Total number of essays containing attainment are: {}".format(attainment_df.shape[0]))

Total number of essays containing attainment are: 185


**Creating dataset for first generation**

In [15]:
first_generation_df = df.copy()

In [16]:
# dropping the attainment column from the dataset.
first_generation_df = first_generation_df.drop(columns=['Attainment - FINAL'])

In [17]:
first_generation_df.shape

(570, 3)

In [18]:
# drop essays which do not contain first generation
first_generation_df = first_generation_df[~first_generation_df['First Gen - FINAL'].isin([0, '-', 'Unable to code'])]

In [19]:
first_generation_df.shape

(20, 3)

In [20]:
first_generation_df = first_generation_df.dropna() # drop null

In [21]:
first_generation_df.shape

(19, 3)

In [22]:
# total number of essays
print("Total number of essays containing first generation thematic code are: {}".format(first_generation_df.shape[0]))

Total number of essays containing first generation thematic code are: 19


**Data Preparation**  
1. Get noun and adjective

In [23]:
def get_noun_adj(phrase):
    noun_adj = []
    for tag in pos_tag(phrase.split()):
        if tag[1] in ['NN','NNP','NNS', 'NNPS','JJ', 'JJR', 'JJS']:
            noun_adj.append(tag[0])
    return  noun_adj

2. Get NER from sentence
    - Need to have the corenlp server up and running

In [50]:
# Need to have corenlp server running
def get_ner(text):
    doc = nlp(text)
    ner_list = [ent.text for sent in doc.sentences for ent in sent.ents]
    return ner_list

In [56]:
sample_sentence = "Barack Obama was born in Hawaii"

get_ner(sample_sentence)

['Barack Obama', 'Hawaii']

3. Get sentence label
    1. Look for exact match
    2. Check for NER in phrase (title/profession/scientific fields) and check if those exist in the sentence
    3. Get the nouns and adjectives and check if those exist in the sentence

In [57]:
def get_sentence_label(essay_sentence, phrase): 
    # 1. look for the exact match of the phrase in the sentence
    match = re.search('.*%s.*' %phrase,essay_sentence, re.IGNORECASE)
    #print(essay_sentence)
    if match:
        return 1
    else:        
        # 2. check for NER in phrase (title /profession/ scientific fields) check if those exist in sentence
        ner_list = get_ner(phrase)
        ner_found_flag = None
        if len(ner_list) == 0:
            ner_found_flag = 0
        else: 
            for ner in ner_list:
                if not (re.search('.*%s.*' %ner, essay_sentence)):
                    ner_found_flag = 0
                    break
                else:
                    ner_found_flag = 1
        if ner_found_flag:
            return 1
        else:
            # 3. Else get the nouns and adjectives and check if those present in the sentence
            noun_adj_phrase = get_noun_adj(phrase)
            all_pos_found = None
            for pos in noun_adj_phrase:
                if pos not in essay_sentence:
                    all_pos_found = 0
                    break
                else:
                    all_pos_found = 1
            if all_pos_found == 1:
                return 1
            else:
                return 0

## Looking at example 1, index 0

In [30]:
# entire essay
attainment_df["Essay : Why am I here"][0]

'I am here to get my BS in Physiology, so I can get into Medical School. For this class, I am trying to make friends/connection with people who are trying to follow the same path. And also POTLUCK!!'

In [31]:
#sentencep positive for attainment
attainment_df["Attainment - FINAL"][0]

'I am here to get my BS in physiology so I can get into medical school.'

**Generate csv file for Attainment**

In [60]:
attainment_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('data/csv/attainment_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in attainment_df.iterrows():
        text = val['Essay : Why am I here'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['Attainment - FINAL']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            # Create a new DataFrame with the row you want to add
            new_row = pd.DataFrame([{'sentence': sentence.lower(), 'label': label, 'phrase': annotation_strip}])

            # Concatenate the new row to the existing DataFrame
            attainment_sentence_df = pd.concat([attainment_sentence_df, new_row], ignore_index=True)
            # attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #attainment_sentence_df = attainment_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)

In [59]:
attainment_df

,ID CODE,Essay : Why am I here,Attainment - FINAL
0,pilot_study_01,"I am here to get my BS in Physiology, so I can...",I am here to get my BS in physiology so I can ...
8,pilot_study_09,I’m here to learn and to be on the path toward...,I’m here to learn and to be on the path toward...
9,pilot_study_10,I am in SFSU because I want to go to medical s...,I am in SFSU because I want to go to medical s...
11,pilot_study_12,I am here in SFSU because I am the first gener...,My goal is to graduate with a degree B.A./B.S.
13,pilot_study_14,I am here because of my goal in trying to get ...,I am here because of my goal in trying to get ...
...,...,...,...
551,F19.PHYS232.07.23,I'm here because of prerequisite for my comput...,And my dream job is to be a software engineer.
559,F19.PHYS232.09.08,The reason I am here is to do experiments and ...,I am an aspiring Engineer and I love doing ex...
561,F19.PHYS232.09.10,I am in college to further my education. There...,I am studying to become an electrical engineer.
562,F19.PHYS242.05.01,I am here at SFSU for a multitude of reasons. ...,My second is to receive a degree in Engineerin...


In [58]:
attainment_sentence_df

,sentence,label,phrase


In [27]:
pos_labels = len([n for n in attainment_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(attainment_sentence_df['label']), (pos_labels/len(attainment_sentence_df['label']))*100))


Positive labels present in the dataset : 253  out of 1413 or 17.905166312809627%


In [28]:
train, test = train_test_split(attainment_sentence_df, test_size=0.1, random_state=18, stratify=attainment_sentence_df['label'])

In [29]:
train.to_csv("data/csv/attainment_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [30]:
test.to_csv("data/csv/attainment_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [31]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 17.94% positive class
positive labels : 228 out of total 1271 datapoints


In [32]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 17.61% positive class
positive labels : 25 out of total 142 datapoints


**Generate csv file for First Generation**

In [33]:
first_generation_sentence_df = pd.DataFrame(columns=['sentence', 'label', 'phrase'])
with open('data/csv/first_generation_sentence_level_dataset.csv','w',newline='', encoding = 'UTF-8') as f:
    writer = csv.writer(f)
    writer.writerow(['sentence','phrases','label'])
    for index,val in first_generation_df.iterrows():
        text = val['Essay : Why am I here'].rstrip('\n\r')
        # using sent_tokenize from nltk to split the paragraphs into sentences.
        temp_text = re.split('\.|\?|\!',text) # text.split('.')
        # annotation
        annotation = val['First Gen - FINAL']
        if '/%/' in annotation:
            annotation = re.sub('/%/', '. ', annotation)
        # annotation list
        annotation_list = re.split('\.|\?|\!',annotation) #annotation.split('.')
        annotation_strip = [y.rstrip('\r\n') for y in annotation_list if y != '']
        essay_strip = [x for x in temp_text if x !='']
        
        # remove the extra spaces before the sentences.
        annotation_strip = [y.lstrip(" ") for y in annotation_strip]
        essay_strip = [x.lstrip(" ") for x in essay_strip]
        
        # annotate the essay on sentence level.
        for sentence in essay_strip:
            if not (re.search('\w',sentence)):
                continue
            if (len(sentence.split(' ')) < 3):
                continue
            for annotation_phrase in annotation_strip:
                if len(annotation_phrase.split()) > 2:
                    label = get_sentence_label(sentence,annotation_phrase)
                    if label == 1:
                        break
            writer.writerow([sentence.lower(), annotation_strip, label])
            first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence.lower() , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            #first_generation_sentence_df = first_generation_sentence_df.append({'sentence' : sentence , 'label': label , 'phrase':annotation_strip}, ignore_index=True)
            

In [34]:
first_generation_sentence_df.shape

(128, 3)

In [35]:
pos_labels = len([n for n in first_generation_sentence_df['label'] if n==1])
print("Positive labels present in the dataset : {}  out of {} or {}%".format(pos_labels, len(first_generation_sentence_df['label']), (pos_labels/len(first_generation_sentence_df['label']))*100))


Positive labels present in the dataset : 19  out of 128 or 14.84375%


In [36]:
train, test = train_test_split(first_generation_sentence_df, test_size=0.2, random_state=18, stratify=first_generation_sentence_df['label'])

In [37]:
train.to_csv("data/csv/first_generation_sentence_level_training_data.csv", encoding='UTF-8', index=False)

In [38]:
test.to_csv("data/csv/first_generation_sentence_level_test_data.csv",  encoding='UTF-8', index=False)

In [39]:
print('Class distibution of Training data: {:0.2f}% positive class'.format(len([m for m in train['label'] if m==1])/len(train['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in train['label'] if m==1]), len(train['label'])))

Class distibution of Training data: 14.71% positive class
positive labels : 15 out of total 102 datapoints


In [40]:
print('Class distibution of Test data: {:0.2f}% positive class'.format(len([m for m in test['label'] if m==1])/len(test['label']) *100))
print('positive labels : {} out of total {} datapoints'.format(len([m for m in test['label'] if m==1]), len(test['label'])))

Class distibution of Test data: 15.38% positive class
positive labels : 4 out of total 26 datapoints
